In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key 

In [ ]:
#import csv and then create heatmap. Look up syuntax from heatnmap actviity.
#Use .loc function in order to filtyer down the df hum<..., etc..
#Create hotel_df and add column called "Hotel Name"
#Run google API calls and after I can refer googledat activity
#From there I can use that to get api calls for the hotel name in those cities
#Use the same heatmap and just add marker layer -- Look in the 3rd day actvoty folder


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
 # Add marker layer ontop of heat map


# Display figure